In [ ]:
import geopandas as gpd

In [ ]:
kanagawa=gpd.read_file("data/A002005212015DDSWC14/h27ka14.shp")

In [ ]:
kanagawa.plot()

In [ ]:
import pandas as pd
pop=pd.read_csv("data/tblT000849C14.txt",encoding="SHIFT-JIS",skiprows=[1],na_values=["X","-"],dtype=str)

In [ ]:
kanagawa[kanagawa["KEY_CODE"]=="144010070"]

In [ ]:
tsunami_kanagawafp="data/A40-16_14_GML/A40-16_14.shp"
tsunami_kanagawa=gpd.read_file(tsunami_kanagawafp)

In [ ]:
kanagawa=kanagawa.to_crs("EPSG:6668")
print(tsunami_kanagawa.crs,kanagawa.crs)
ax=kanagawa.plot(alpha=0.2,figsize=(40,40))
#ax=kanagawa.boundary.plot()
tsunami_kanagawa.plot(ax=ax,color="red")

In [ ]:
from geopy.geocoders import Nominatim
from shapely.geometry import Point

In [ ]:
ninomiya_evac_fac=pd.read_csv("data/ninomiya_fac.txt")
geolocator = Nominatim(user_agent="test-dayo")
ninomiya_evac_fac['gcode']=ninomiya_evac_fac.addr.apply(geolocator.geocode)

In [ ]:
ninomiya_evac_fac=pd.read_csv("data/ninomiya_fac.txt")
ninomiya_evac_fac.addr
ninomiya_evac_fac['gcode']=ninomiya_evac_fac.addr.apply(geolocator.geocode)

In [ ]:
ninomiya_evac_fac=pd.read_csv("data/ninomiya_fac.txt")

In [ ]:
ninomiya_evac_fac

In [ ]:
#ninomiya_evac_fac
from shapely.geometry import Point
evac_fac=gpd.GeoDataFrame()
evac_pt=[]
for i,d in ninomiya_evac_fac.iterrows():
    print(i,Point(d.lat,d.lon))
    evac_pt+=[Point(d.lon,d.lat)]
print(evac_pt)
#evac_fac=gpd.GeoDataFrame(geometry=evac_pt)
evac_fac=gpd.GeoDataFrame(geometry=evac_pt)
evac_fac['ninzu']=ninomiya_evac_fac['ninzu']
evac_fac['name']=ninomiya_evac_fac['name']
evac_fac=evac_fac.set_crs("EPSG:6668")
evac_fac

In [ ]:
evac_fac['name'] = None
for index, row in ninomiya_evac_fac.iterrows():
    evac_fac.loc[index, 'name'] = row["name"]

In [ ]:
ninomiya=kanagawa[ kanagawa["CITY_NAME"]=="二宮町"]
ninomiya = ninomiya.reset_index(drop=True)
print(ninomiya.crs,evac_fac.crs)
ax=ninomiya.plot(alpha=0.2,figsize=(40,40))
tsunami_kanagawa.plot(ax=ax,color="red")
evac_fac.plot(ax=ax,color="black")

In [ ]:
#ninomiya=ninomiya.to_crs("EPSG:6668")

In [ ]:
tsunami_ninomiya=gpd.sjoin(tsunami_kanagawa,ninomiya,how='inner',op='intersects')

In [ ]:
ax=ninomiya.boundary.plot(alpha=0.5,figsize=(40,40))
tsunami_ninomiya.plot(ax=ax,color="red")
evac_fac.plot(ax=ax,color="black")

In [ ]:
for i,row in ninomiya.iterrows():
    print(row['geometry'].centroid)

In [ ]:
import osmnx as ox
place_name = "Ninomiya, Kanagawa"
ninomiya_graph = ox.graph_from_place(place_name)

In [ ]:
ox.plot_graph(ninomiya_graph,node_size=0)

In [ ]:
# impute missing edge speeds and calculate edge travel times with the speed module
ninomiya_graph = ox.speed.add_edge_speeds(ninomiya_graph)
ninomiya_graph = ox.speed.add_edge_travel_times(ninomiya_graph)

In [ ]:
ninomiya

In [ ]:
# get the nearest network nodes to two lat/lng points with the distance module
#orig = ox.distance.nearest_nodes(G, X=-122.245846, Y=37.828903)
#dest = ox.distance.nearest_nodes(G, X=-122.215006, Y=37.812303)
#最初の基本区のcentroid
#for i,row in ninomiya.iterrows():
#    print(row['geometry'].centroid)
#type(ninomiya)


In [ ]:
n0=ninomiya.loc[0].geometry.centroid #最初の基本区
orig = ox.distance.nearest_nodes(ninomiya_graph, X=n0.x, Y=n0.y) #n0に最も近いノードを求める
e0=evac_fac.loc[0].geometry #最初の広域避難所
dest=ox.distance.nearest_nodes(ninomiya_graph,X=e0.x,Y=e0.y)#e0に最も近いノードを求める

In [ ]:
# find the shortest path between nodes, minimizing travel time, then plot it
route = ox.shortest_path(ninomiya_graph, orig, dest, weight="travel_time")
fig, ax = ox.plot_graph_route(ninomiya_graph, route, node_size=0)

In [ ]:
import matplotlib.pyplot as plt
fig, ax= plt.subplots(1,1, figsize=(40,10))
ninomiya.boundary.plot(ax=ax,alpha=0.5,figsize=(40,40))
tsunami_ninomiya.plot(ax=ax,color="red")
evac_fac.plot(ax=ax,color="black")
#print(ninomiya.crs,tsunami_ninomiya.crs,evac_fac.crs)

In [ ]:
#グラフを，geodataframeに変換して，nodes,edgesに保存する。
nodes,edges = ox.graph_to_gdfs(ninomiya_graph, nodes=True, edges=True)

In [ ]:
edges['geometry'].to_crs("EPSG:6668")
nodes['geometry'].to_crs("EPSG:6668")

In [ ]:
import matplotlib.pyplot as plt
fig, ax= plt.subplots(1,1, figsize=(40,10))
ninomiya.boundary.plot(ax=ax,alpha=0.5,figsize=(40,40))
tsunami_ninomiya.plot(ax=ax,color="red")
evac_fac.plot(ax=ax,color="black")
edges['geometry'].plot(ax=ax)

In [ ]:
orig_pt,dest_pt=nodes.at[orig,"geometry"],nodes.at[dest,"geometry"]

od_points = gpd.GeoDataFrame(crs=edges.crs)
od_points['geometry'] = None
od_points['type'] = None
od_points.loc[0, ['geometry', 'type']] = orig_pt, 'Origin'
od_points.loc[1, ['geometry', 'type']] = dest_pt, 'Target'
od_points['geometry'].to_crs("EPSG:6668")

In [ ]:
fig, ax= plt.subplots(1,1, figsize=(40,40))
ninomiya.boundary.plot(ax=ax,alpha=0.5,figsize=(40,40))
tsunami_ninomiya.plot(ax=ax,color="red")
evac_fac.plot(ax=ax,color="black")
edges['geometry'].plot(ax=ax)
od_points.plot(ax=ax,color="#ff7f00")

In [ ]:
#od_points.plot()

>本研究では，解析範囲内の国勢調査の基本単位区毎に
累積収容人数を累積収容人数曲線として表現し，基本単
位区毎の累積収容人数を比較した．

各基本単位区から，各広域避難所までの移動距離を求める。

In [ ]:
ku_dist=pd.DataFrame()
for i,row in ninomiya.iterrows():
    #i番目の基本区のcentroidをポイントとしてcent_locに設定する
    print(i,"番目の基本区から")
    cent_loc=Point(row['geometry'].centroid.x,row['geometry'].centroid.y)
    orig=ox.distance.nearest_nodes(ninomiya_graph,X=cent_loc.x,Y=cent_loc.y)
    print("orig:",orig)
    for j,row2 in evac_fac.iterrows():
        #j番目の施設をポイントとしてfac_locに設定する
        fac_loc=Point(row2['geometry'].x,row2['geometry'].y)
        dest=ox.distance.nearest_nodes(ninomiya_graph,X=fac_loc.x,Y=fac_loc.y)
        print("dest:",dest)
        
        route = ox.shortest_path(ninomiya_graph, orig, dest, weight="travel_time")

        
    print("")
    print("")


In [ ]:
evac_fac

In [ ]:
#    print("")
import networkx as nx
ku_dist=pd.DataFrame()
for i,row in ninomiya.iterrows():
    #i番目の基本区のcentroidをポイントとしてcent_locに設定する
    print(i,"番目の基本区から")
    cent_loc=Point(row['geometry'].centroid.x,row['geometry'].centroid.y)
    ku_dist.loc[i,"cent_loc"]=row['KEY_CODE']
    orig=ox.distance.nearest_nodes(ninomiya_graph,X=cent_loc.x,Y=cent_loc.y)
    for j,row2 in evac_fac.iterrows():
        fac_loc=Point(row2['geometry'].x,row2['geometry'].y)
        dest=ox.distance.nearest_nodes(ninomiya_graph,X=fac_loc.x,Y=fac_loc.y)
        print("cent_loc:",cent_loc.x,cent_loc.y," fac:",fac_loc.x,fac_loc.y)
        route = ox.shortest_path(ninomiya_graph, orig, dest)
        min_dist = nx.shortest_path_length(ninomiya_graph, orig, dest)
        ku_dist.loc[i,j]=min_dist*100
        
        print("dist:",min_dist*100, " m")
        fig,ax=ox.plot_graph_route(ninomiya_graph,route,node_size=0)

In [ ]:
#各基本区のcentroidから，各広域避難所への距離
ku_dist
#平均徒歩避難速度を50m/分として， 一定時間以内に到達可能な広域避難所の収容人数を足して累積収容人数を計算する。

In [ ]:
thres_min=[5,10,15,20,30] #累積収容人数を計算するための移動時間閾値
walk_speed=50
for i,row in  ku_dist.iterrows():
    print(i,"th 基本区")
    row['cum_cap']=0
    for tmin in thres_min:
        print(tmin," 分以内に到達できる施設")
        print(row[ row.astype(float)/walk_speed< tmin ] )
        fac_reachable=row.astype(float)/walk_speed< tmin
        print("fac:",fac_reachable)
#        print("sum:",row[ row.astype(float)/walk_speed< tmin ] )
        print("fac reachable:",ninomiya_evac_fac[fac_reachable])
#        print("cumsum:",ninomiya_evac_fac[fac_reachable]['ninzu'].sum())
        #row['cum_cap'+str(tmin)]=ninomiya_evac_fac[fac_reachable]['ninzu'].sum()
        print("")
#print(ku_dist)        